In [32]:
# Kor!
# from kor.extraction import create_extraction_chain
# from kor.nodes import Object, Text, Number

# LangChain Models
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.chains import create_extraction_chain
# Standard Helpers
import pandas as pd
import requests
import time
import json
from datetime import datetime

# Text Helpers
from bs4 import BeautifulSoup
from markdownify import markdownify as md

# For token counting
from langchain.callbacks import get_openai_callback

def printOutput(output):
    print(json.dumps(output,sort_keys=True, indent=3))

In [18]:
openai_api_key = ""


In [29]:
llm = ChatOpenAI(
    # model_name="gpt-3.5-turbo",
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=2000,
    openai_api_key=openai_api_key
)

In [20]:
jd="""JOB DESCRIPTION / ROLES

F1Soft Group has been building an ecosystem of digital products and services to enable Digital Financial Transformation. From its inception, F1Soft has been committed to making easier, faster, and more convenient digital financial solutions that can help end users feel more empowered and connected. With innovation and technology at the core of everything we do, our products and services can help take businesses to the next level. We believe in providing effective and efficient digital financial solutions that enable access to a digital economy.

RESPONSIBILITIES

· Analyse and understand business requirements in details
· Work in Scrum/Kanban team environment to size and plan the requirements
· Develop software products following standard coding practices and automated unit testing
· Maintain software products by identifying and resolving issues
· Work collaboratively in an Agile team to deliver team commitments and goals
· Maintain required documentation of own's work
· Research on new technology and techniques for continuous improvement of the products
· Participate actively in retrospective meetings for continuous improvement of the team


REQUIREMENT

· Bachelors Degree in Computer Science, Computer Engineering or related technical Discipline
· Minimum 1 to 3 Years of prior work experience in related field
· Familiar with Object Oriented Concept
· Familiar with Data Structure and DBMS
· Familiar with Agile/Scrum Practices and Tools


 Required Skills

· JAVA
· Spring Framework
· Database (MySQL)
"""

In [21]:
tools=Object(
id="tools",
description="""Skills required in Job Description""",
attributes=[
    Text(
    id="tool",
        description="Name of the technology skill, tool or a company"
    )
],
    examples=[
        (
            "Experience in working with Netsuite, or Looker a plus.",
            [
                {"tool": "Netsuite"},
                {"tool": "Looker"},
            ],
        ),
        (
           "Experience with Microsoft Excel",
            [
               {"tool": "Microsoft Excel"}
            ] 
        ),
        (
           "You must know AWS to do well in the job",
            [
               {"tool": "AWS"}
            ] 
        ),
        (
           "Troubleshooting customer issues and debugging from logs (Splunk, Syslogs, etc.) ",
            [
               {"tool": "Splunk"},
            ] 
        )
    ],
    many=True,
)

In [25]:
chain = create_extraction_chain(llm, tools, input_formatter="triple_quotes")


In [26]:
output = chain.predict_and_parse(text=jd)["data"]

printOutput(output)

/home/sashank/miniconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


TypeError: initial_value must be str or None, not dict

In [35]:
schema={"properties":{"responsibilities":{"type":"string"},"requirements":{"type":"string"},'tools/skills':{"type":"string"}}}

In [36]:
chain=create_extraction_chain(schema,llm)

In [37]:
chain.run(jd)

[{'responsibilities': "Analyse and understand business requirements in details\nWork in Scrum/Kanban team environment to size and plan the requirements\nDevelop software products following standard coding practices and automated unit testing\nMaintain software products by identifying and resolving issues\nWork collaboratively in an Agile team to deliver team commitments and goals\nMaintain required documentation of own's work\nResearch on new technology and techniques for continuous improvement of the products\nParticipate actively in retrospective meetings for continuous improvement of the team",
  'requirements': 'Bachelors Degree in Computer Science, Computer Engineering or related technical Discipline\nMinimum 1 to 3 Years of prior work experience in related field\nFamiliar with Object Oriented Concept\nFamiliar with Data Structure and DBMS\nFamiliar with Agile/Scrum Practices and Tools',
  'tools/skills': 'JAVA\nSpring Framework\nDatabase (MySQL)'}]